In [1]:
%matplotlib inline

from pyhandy import get_data_summary
import pandas as pd
import glob

from src.preprocessing import preprocess_madrid_aq_data as preprocess
from src.constants import indicators_code_dict, indicators_abbrev_dict, estaciones_codes_dict

### Datos Meteorologicos

In [21]:
meteo_csvs = glob.glob("../00-rawdata/Ayto Madrid/Meteo Ayto Madrid/*.csv", recursive = True) #Simon
print(len(meteo_csvs),'datasets de datos meteorologicos cubriendo 18 meses desde Enero de 2019 hasta agosto de 2021')

32 datasets de datos meteorologicos cubriendo 18 meses desde Enero de 2019 hasta agosto de 2021


In [42]:
meteo_dfs, num_rows = [], 0
for csv in meteo_csvs:
    df = pd.read_csv(csv,sep=';',decimal=',')
    meteo_dfs.append(df)
    num_rows += len(df)

print(len(meteo_dfs),'csv cargados')
print(num_rows,'observaciones en total')
meteo_df = pd.concat(meteo_dfs)
meteo_df #Asi vienen los datos meteorologicos de Ayto. Madrid

32 csv cargados
87601 observaciones en total


,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,102,81,28079102_81_98,2021,5,1,00.97,V,...,01.67,V,01.37,V,00.93,V,00.85,V,01.10,V
1,28,79,102,81,28079102_81_98,2021,5,2,02.23,V,...,01.38,V,01.93,V,02.12,V,01.70,V,02.57,V
2,28,79,102,81,28079102_81_98,2021,5,3,02.70,V,...,03.32,V,01.27,V,02.60,V,01.92,V,02.75,V
3,28,79,102,81,28079102_81_98,2021,5,4,02.25,V,...,01.78,V,01.57,V,01.67,V,00.75,V,00.38,V
4,28,79,102,81,28079102_81_98,2021,5,5,01.00,V,...,02.05,V,01.62,V,01.57,V,01.18,V,00.68,V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,28,79,59,89,28079059_89_98,2021,4,26,00.00,V,...,00.00,V,00.00,V,00.00,V,00.00,V,01.50,V
2696,28,79,59,89,28079059_89_98,2021,4,27,01.20,V,...,00.00,V,00.00,V,03.80,V,03.00,V,02.10,V
2697,28,79,59,89,28079059_89_98,2021,4,28,00.10,V,...,00.00,V,00.00,V,00.00,V,00.00,V,00.00,V
2698,28,79,59,89,28079059_89_98,2021,4,29,00.00,V,...,00.00,V,00.00,V,00.00,V,00.00,V,00.00,V


Del Interprete de Ficheros Meteorologicos dado por el ayuntamiento de Madrid vemos que:

| Codigo | Parametro | Unidad | Tecnica de medida |
| --- | --- | --- | --- |
| 80 | RADIACION ULTRAVIOLETA | Mw/m2 | 98 |
| 81 | VELOCIDAD VIENTO | m/s | 98 |
| 82 | DIR. DE VIENTO | º | 98 |
| 83 | TEMPERATURA | ºC | 98 |
| 86 | HUMEDAD RELATIVA | % | 98 |
| 87 | PRESION BARIOMETRICA | mb | 98 |
| 88 | RADIACION SOLAR | W/m2 | 98 |
| 89 | PRECIPITACIÓN | l/m2 | 98 |

In [43]:
parameters_dict = { #from docs
    80:{'parametro':'RADIACION ULTRAVIOLETA','unidad':'Mw/m2','tecnica_de_medida':98},
    81: {'parametro':'VELOCIDAD VIENTO','unidad':'m/s','tecnica_de_medida':98},
    82 : {'parametro':'DIR. DE VIENTO','unidad':'º','tecnica_de_medida':98},
    83 : {'parametro':'TEMPERATURA','unidad':'ºC','tecnica_de_medida':98},
    86 : {'parametro':'HUMEDAD RELATIVA','unidad':'%','tecnica_de_medida':98},
    87 : {'parametro':'PRESION BARIOMETRICA','unidad':'mb','tecnica_de_medida':98},
    88 : {'parametro':'RADIACION SOLAR','unidad':'W/m2','tecnica_de_medida':98},
    89 : {'parametro':'PRECIPITACIÓN','unidad':'l/m2','tecnica_de_medida':98}
}

In [44]:
meteo_df['PARAMETRO'] = meteo_df.PUNTO_MUESTREO.apply(lambda x: parameters_dict[int(x.split('_')[1])]['parametro'])
meteo_df['UNIDAD'] = meteo_df.PUNTO_MUESTREO.apply(lambda x: parameters_dict[int(x.split('_')[1])]['unidad'])
meteo_df['FECHA'] = meteo_df.ANO.astype(str) + '-' + meteo_df.MES.astype(str).str.zfill(2) + '-' + meteo_df.DIA.astype(str).str.zfill(2)
measurement_cols = meteo_df.columns[meteo_df.columns.str.startswith('H')]
meteo_df_clean = meteo_df[['FECHA','PROVINCIA', 'MUNICIPIO', 'ESTACION', 'MAGNITUD','PARAMETRO','UNIDAD']+measurement_cols.tolist()]

In [45]:
df = meteo_df_clean.melt(['FECHA','PROVINCIA', 'MUNICIPIO', 'ESTACION', 'MAGNITUD','PARAMETRO','UNIDAD']).sort_values('FECHA').rename(columns={'variable':'TIME'})
df['value'] = df['value'].astype(float)
df['TIME'] = ''+df['TIME'].str.replace('H','').str.replace('24','00')+':00:00'
df['FECHA'] = pd.to_datetime(df['FECHA'] + ' ' + df['TIME'])
df = df.drop(columns='TIME')
df.columns=df.columns.str.lower().tolist()
df #Podemos preprocesar los datos para que se vean asi, donde el parametro y su respectivo valor esta cada uno en una columna de la misma fila...

,fecha,provincia,municipio,estacion,magnitud,parametro,unidad,value
1736660,2019-01-01 20:00:00,28,79,107,89,PRECIPITACIÓN,l/m2,0.0
1825216,2019-01-01 21:00:00,28,79,24,86,HUMEDAD RELATIVA,%,63.0
774283,2019-01-01 09:00:00,28,79,38,86,HUMEDAD RELATIVA,%,58.0
862628,2019-01-01 10:00:00,28,79,59,88,RADIACION SOLAR,W/m2,110.0
2087554,2019-01-01 00:00:00,28,79,113,83,TEMPERATURA,ºC,5.9
...,...,...,...,...,...,...,...,...
1594175,2021-08-31 19:00:00,28,79,107,82,DIR. DE VIENTO,º,75.0
281642,2021-08-31 04:00:00,28,79,58,83,TEMPERATURA,ºC,17.0
1857964,2021-08-31 22:00:00,28,79,38,83,TEMPERATURA,ºC,28.6
1944827,2021-08-31 23:00:00,28,79,110,83,TEMPERATURA,ºC,27.3


In [46]:
get_data_summary(df) #summary

Dataset has 2102424 rows and 8 columns


,DataType,NumMissing,Mean,Median,Mode,MinValue,MaxValue,NumOfUnique,UniqueValues,FracUnique
ColumnName,,,,,,,,,,
fecha,datetime64[ns],0,NaN,NaN,NaN,2019-01-01 00:00:00,2021-08-31 23:00:00,23376,NaN,NaN
value,float64,0,132.958794,23.1,0.0,-55.0,3789.0,2446,NaN,NaN
estacion,int64,0,77.271310,102.0,24,4,115,26,NaN,NaN
magnitud,int64,0,84.845789,86.0,86,80,89,8,NaN,NaN
municipio,int64,0,79.000000,79.0,79,79,79,1,NaN,NaN
provincia,int64,0,28.000000,28.0,28,28,28,1,NaN,NaN
parametro,object,0,NaN,NaN,HUMEDAD RELATIVA,NaN,NaN,8,"[DIR. DE VIENTO, HUMEDAD RELATIVA, PRECIPITACI...","[0.247, 0.246, 0.11, 0.109, 0.109, 0.088, 0.08..."
unidad,object,0,NaN,NaN,%,NaN,NaN,8,"[%, Mw/m2, W/m2, l/m2, m/s, mb, º, ºC]","[0.247, 0.246, 0.11, 0.109, 0.109, 0.088, 0.08..."


In [47]:
df_ = df.copy()
df_['parametro'] = (df_['parametro']+' ('+df_['unidad']+')').str.lower()
df_ = df_.set_index(['fecha','provincia','municipio','estacion','parametro'])['value'].unstack().reset_index()
df_.columns=df_.columns.str.lower().tolist()
df_ #O asi, donde cada parametro es una columna. Este me parece mas interpretable. El problema es que habran muchos NaNs <--------

,fecha,provincia,municipio,estacion,dir. de viento (º),humedad relativa (%),precipitación (l/m2),presion bariometrica (mb),radiacion solar (w/m2),radiacion ultravioleta (mw/m2),temperatura (ºc),velocidad viento (m/s)
0,2019-01-01 00:00:00,28,79,4,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1,2019-01-01 00:00:00,28,79,8,NaN,58.0,NaN,NaN,NaN,NaN,6.4,NaN
2,2019-01-01 00:00:00,28,79,16,NaN,61.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01 00:00:00,28,79,18,NaN,NaN,NaN,NaN,NaN,NaN,2.2,NaN
4,2019-01-01 00:00:00,28,79,24,33.0,87.0,0.0,957.0,1.0,1.0,-0.2,0.52
...,...,...,...,...,...,...,...,...,...,...,...,...
585787,2021-08-31 23:00:00,28,79,109,NaN,43.0,NaN,NaN,NaN,NaN,27.2,NaN
585788,2021-08-31 23:00:00,28,79,110,NaN,46.0,NaN,NaN,NaN,NaN,27.3,NaN
585789,2021-08-31 23:00:00,28,79,112,NaN,45.0,NaN,NaN,NaN,NaN,27.9,NaN
585790,2021-08-31 23:00:00,28,79,113,NaN,44.0,NaN,NaN,NaN,NaN,28.4,NaN


In [48]:
get_data_summary(df_) #No hay casi datos para la mayoria de parametros a excepcion de los de temperatura y humedad relativa

Dataset has 585792 rows and 12 columns


,DataType,NumMissing,Mean,Median,Mode,MinValue,MaxValue,NumOfUnique
ColumnName,,,,,,,,
fecha,datetime64[ns],0,NaN,NaN,NaN,2019-01-01 00:00:00,2021-08-31 23:00:00,23376
dir. de viento (º),float64,357672,146.906128,146.00,0.0,0.0,360.0,361
humedad relativa (%),float64,65928,53.998409,53.00,44.0,-25.0,100.0,110
precipitación (l/m2),float64,355128,0.039216,0.00,0.0,0.0,30.4,136
presion bariometrica (mb),float64,402600,940.180052,944.00,942.0,0.0,1137.0,162
radiacion solar (w/m2),float64,401736,201.664944,8.00,0.0,0.0,3789.0,1075
radiacion ultravioleta (mw/m2),float64,575712,15.830258,1.00,1.0,0.0,199.0,186
temperatura (ºc),float64,68160,15.686995,14.50,0.0,-55.0,61.9,712
velocidad viento (m/s),float64,356976,1.363074,1.13,0.0,0.0,10.8,776


In [49]:
get_data_summary(df_[['fecha','municipio','estacion','temperatura (ºc)','humedad relativa (%)']].dropna(how='any')) #Esto se puedan usar

Dataset has 473136 rows and 5 columns


,DataType,NumMissing,Mean,Median,Mode,MinValue,MaxValue,NumOfUnique
ColumnName,,,,,,,,
fecha,datetime64[ns],0,NaN,NaN,NaN,2019-01-01 00:00:00,2021-08-31 23:00:00,23376
humedad relativa (%),float64,0,53.641555,52.0,44.0,-25.0,100.0,109
temperatura (ºc),float64,0,15.602802,14.4,0.0,-55.0,61.9,710
estacion,int64,0,78.944405,103.0,36.0,8,115,22
municipio,int64,0,79.000000,79.0,79.0,79,79,1


### Calidad de Aire Ayto Madrid

In [2]:
import os, zipfile, glob
import pandas as pd
zips = glob.glob("00-rawdata/Ayto Madrid/Anio*.zip", recursive = True)
aq_dfs = []
num_rows = 0
print(len(zips),"carpetas comprimidas contienen datos de calidad de aire")
#Iterar sobre todas las carpetas comprimidas para cargar los csv donde estan los datos
for zip in zips:
    with zipfile.ZipFile(zip) as z:
        for filename in z.namelist():
            if not os.path.isdir(filename):
              # read the file
              if filename.endswith('.csv'):
                df = pd.read_csv(z.open(filename),sep=';',decimal=',') #cargar csv como pandas.Dataframe
                num_rows += len(df)
                aq_dfs.append(df)
print(len(aq_dfs),'csv cargados')
print(num_rows,'observaciones en total')

21 carpetas comprimidas contienen datos de calidad de aire
247 csv cargados
892584 observaciones en total


In [3]:
aq_dfs[0] #Sin procesar

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,4,1,28079004_1_38,2021,1,1,00006,V,...,00006,V,00005,V,00005,V,00005,V,00004,V
1,28,79,4,1,28079004_1_38,2021,1,2,00004,V,...,00007,V,00006,V,00006,V,00006,V,00005,V
2,28,79,4,1,28079004_1_38,2021,1,3,00006,V,...,00007,V,00006,V,00006,V,00006,V,00005,V
3,28,79,4,1,28079004_1_38,2021,1,4,00005,V,...,00006,V,00006,V,00005,V,00005,V,00005,V
4,28,79,4,1,28079004_1_38,2021,1,5,00004,V,...,00007,V,00009,V,00008,V,00006,V,00005,V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4206,28,79,60,14,28079060_14_6,2021,1,27,22.86,V,...,05.61,V,03.37,V,05.77,V,10.36,V,11.63,V
4207,28,79,60,14,28079060_14_6,2021,1,28,15.56,V,...,06.34,V,06.82,V,07.71,V,13.06,V,17.15,V
4208,28,79,60,14,28079060_14_6,2021,1,29,25.75,V,...,07.44,V,02.02,V,01.71,V,16.57,V,25.72,V
4209,28,79,60,14,28079060_14_6,2021,1,30,29.68,V,...,71.91,V,72.82,V,68.85,V,73.54,V,83.13,V


Del Interprete de Ficheros de Calidad de Aire dado por el ayuntamiento de Madrid vemos que:

|Código|parametro|Abreviatura|Unidad de medida|Código técnica de medida|Técnica de medida|
|---|---|---|---|---|---|
|1|Dióxido de Azufre|SO2|µg/m 3|38|Fluorescencia ultravioleta|
|6|Monóxido de Carbono|CO|mg/m3|48|Absorción infrarroja|
|7|Monóxido de Nitrógeno|NO|µg/m 3|8|Quimioluminiscencia|
|8|Dióxido de Nitrógeno|NO2|µg/m 3|8|Id.|
|9|Partículas < 2.5 µm|PM2.5|µg/m3|47|Microbalanza|
|10|Partículas < 10 µm|PM10|µg/m3|47|Id.|
|12|Óxidos de Nitrógeno|NOx|µg/m3|8|Quimioluminiscencia|
|14|Ozono|O3|µg/m 3|6|Absorción ultravioleta|
|20|Tolueno|TOL|µg/m3|59|Cromatografía de gases|
|30|Benceno|BEN|µg/m3|59|Id.|
|35|Etilbenceno|EBE|µg/m3|59|Id.|
|37|Metaxileno|MXY|µg/m3|59|Id.|
|38|Paraxileno|PXY|µg/m3|59|Id.|
|39|Ortoxileno|OXY|µg/m3|59|Id.|
|42|Hidrocarburos totales|TCH|mg/m3|2|Ionización de llama|
|43|Metano|CH4|mg/m3|2|Id.|
|44|Hidrocarburos no metánicos (hexano) |NMHC|mg/m3|2|Id.|

In [4]:
#Preprocesado
for i in indicators_code_dict:
    if indicators_code_dict[i]['parametro'] in indicators_abbrev_dict:
        indicators_code_dict[i]['parametro'] = indicators_abbrev_dict[indicators_code_dict[i]['parametro']]

calidad_aire_df = preprocess(pd.concat(aq_dfs),indicators_code_dict)#.interpolate(limit=6)
calidad_aire_df.columns = calidad_aire_df.columns.str.replace(' |\/','_',regex=True).str.replace('[(|)]|\.','',regex=True).str.replace('m_3','m3',regex=True)
calidad_aire_df['codigo'] = calidad_aire_df.estacion.astype(int).copy()
calidad_aire_df['estacion'] = calidad_aire_df.estacion.astype(int).replace(estaciones_codes_dict).astype(str)
calidad_aire_df

,fecha,provincia,municipio,estacion,ben_µg_m3,ch4_mg_m3,co_mg_m3,ebe_µg_m3,nmhc_mg_m3,no_µg_m3,no2_µg_m3,nox_µg_m3,o3_µg_m3,pm10_µg_m3,pm25_µg_m3,so2_µg_m3,tch_mg_m3,tol_µg_m3,codigo
0,2001-01-01 00:00:00,28,79,Pza. de España,NaN,NaN,0.3,NaN,NaN,5.0,54.0,61.0,NaN,NaN,NaN,13.0,NaN,NaN,4
11,2001-01-01 00:00:00,28,79,Vallecas,NaN,NaN,NaN,NaN,NaN,2.0,19.0,23.0,NaN,4.0,NaN,5.0,NaN,NaN,40
10,2001-01-01 00:00:00,28,79,Barrio del Pilar,NaN,NaN,0.4,NaN,NaN,8.0,19.0,32.0,54.88,NaN,NaN,NaN,NaN,NaN,39
9,2001-01-01 00:00:00,28,79,Cuatro Caminos,NaN,NaN,NaN,NaN,NaN,5.0,19.0,26.0,NaN,5.0,NaN,23.0,NaN,NaN,38
7,2001-01-01 00:00:00,28,79,Pza. del Carmen,NaN,NaN,0.2,NaN,NaN,3.0,22.0,27.0,62.42,NaN,NaN,17.0,NaN,NaN,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3378409,2021-07-31 23:00:00,28,79,Escuelas Aguirre,0.2,NaN,0.2,0.2,NaN,2.0,15.0,18.0,50.81,7.0,2.0,5.0,NaN,0.6,8
3378408,2021-07-31 23:00:00,28,79,Pza. de España,NaN,NaN,0.2,NaN,NaN,1.0,8.0,10.0,NaN,NaN,NaN,9.0,NaN,NaN,4
3378430,2021-07-31 23:00:00,28,79,Parque Juan Carlos I,NaN,NaN,NaN,NaN,NaN,1.0,7.0,9.0,47.27,NaN,NaN,NaN,NaN,NaN,59
3378418,2021-07-31 23:00:00,28,79,Cuatro Caminos,0.1,NaN,NaN,0.1,NaN,2.0,10.0,13.0,NaN,8.0,4.0,NaN,NaN,0.3,38


In [8]:
aq_ayto_summ = get_data_summary(calidad_aire_df) #Summary
aq_ayto_summ

Dataset has 3378432 rows and 18 columns


,DataType,MissingPercent,Mean,Median,Mode,MinValue,MaxValue,NumOfUnique,UniqueValues,FracUnique
ColumnName,,,,,,,,,,
fecha,datetime64[ns],0.000000,NaN,NaN,NaN,2001-01-01 00:00:00,2021-07-31 23:00:00,180408,NaN,NaN
ben_µg_m3,float64,0.782159,0.699115,0.30,0.2,0.0,799.9,409,NaN,NaN
ch4_mg_m3,float64,0.848801,1.231341,1.22,1.22,0.0,350.7,423,NaN,NaN
co_mg_m3,float64,0.526782,0.426379,0.30,0.2,0.0,35.6,132,NaN,NaN
ebe_µg_m3,float64,0.783438,0.785463,0.40,0.1,0.0,1466.0,554,NaN,NaN
nmhc_mg_m3,float64,0.848907,0.164456,0.13,0.12,0.0,9.07,239,NaN,NaN
no2_µg_m3,float64,0.000085,41.385195,34.00,13.0,0.0,9108.0,421,NaN,NaN
no_µg_m3,float64,0.000085,26.503183,7.00,1.0,0.0,86812.0,1079,NaN,NaN
nox_µg_m3,float64,0.000085,81.849946,46.00,15.0,0.0,12957.0,1765,NaN,NaN


In [13]:
#Guardar estos datos en formato feather (antes de eliminar columnas o hacer imputado)
(calidad_aire_df
 .drop(columns=['provincia','municipio'])
 .melt(id_vars=['fecha','estacion'],var_name='variable')
 .dropna(subset=['value'])
 .reset_index(drop=True)
 .to_feather('01-data/interim/aq_ayto_madrid-sin-imputar.feather')
)

In [24]:
from dateutil.parser import parse
date = parse('1-1-2013')
calidad_aire_df_2013 = calidad_aire_df[calidad_aire_df["fecha"] >= date]
print(calidad_aire_df_2013.shape)

(1799640, 18)


In [27]:
(calidad_aire_df_2013
 .drop(columns=['provincia','municipio'])
 .reset_index(drop=True)
 .to_feather('01-data/aq_ayto_madrid-sin-imputar-2013-raw.feather')
)

In [20]:
(calidad_aire_df_2013
 .drop(columns=['provincia','municipio'])
 .melt(id_vars=['fecha','estacion'],var_name='variable')
 .dropna(subset=['value'])
 .reset_index(drop=True)
 .to_feather('01-data/aq_ayto_madrid-sin-imputar-2013.feather')
)

In [28]:
calidad_aire_df_2013

,fecha,provincia,municipio,estacion,ben_µg_m3,ch4_mg_m3,co_mg_m3,ebe_µg_m3,nmhc_mg_m3,no_µg_m3,no2_µg_m3,nox_µg_m3,o3_µg_m3,pm10_µg_m3,pm25_µg_m3,so2_µg_m3,tch_mg_m3,tol_µg_m3
1578806,2013-01-01 00:00:00,28,79,Pº. Castellana,NaN,NaN,NaN,NaN,NaN,18.0,58.0,85.0,NaN,9.0,6.0,NaN,NaN,NaN
1578807,2013-01-01 00:00:00,28,79,Retiro,NaN,NaN,NaN,NaN,NaN,29.0,46.0,91.0,1.48,NaN,NaN,NaN,NaN,NaN
1578808,2013-01-01 00:00:00,28,79,Pza. Castilla,NaN,NaN,NaN,NaN,NaN,51.0,67.0,146.0,NaN,18.0,11.0,NaN,NaN,NaN
1578809,2013-01-01 00:00:00,28,79,Ensanche Vallecas,NaN,NaN,NaN,NaN,NaN,21.0,44.0,76.0,4.81,NaN,NaN,NaN,NaN,NaN
1578810,2013-01-01 00:00:00,28,79,Urb. Embajada (Barajas),1.8,1.63,NaN,0.9,0.33,121.0,70.0,255.0,NaN,22.0,NaN,NaN,1.96,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3378409,2021-07-31 23:00:00,28,79,Escuelas Aguirre,0.2,NaN,0.2,0.2,NaN,2.0,15.0,18.0,50.81,7.0,2.0,5.0,NaN,0.6
3378408,2021-07-31 23:00:00,28,79,Pza. de España,NaN,NaN,0.2,NaN,NaN,1.0,8.0,10.0,NaN,NaN,NaN,9.0,NaN,NaN
3378430,2021-07-31 23:00:00,28,79,Parque Juan Carlos I,NaN,NaN,NaN,NaN,NaN,1.0,7.0,9.0,47.27,NaN,NaN,NaN,NaN,NaN
3378418,2021-07-31 23:00:00,28,79,Cuatro Caminos,0.1,NaN,NaN,0.1,NaN,2.0,10.0,13.0,NaN,8.0,4.0,NaN,NaN,0.3


In [32]:
pd.read_feather('01-data/aq_ayto_madrid-sin-imputar_2013.feather') #Los datos se guardan en este formato para ahorrar espacio de almacenamiento

,fecha,estacion,variable,value
0,2001-01-01 00:00:00,Escuelas Aguirre,ben_µg_m3,3.8
1,2001-01-01 00:00:00,Casa de Campo,ben_µg_m3,0.2
2,2001-01-01 01:00:00,Escuelas Aguirre,ben_µg_m3,4.1
3,2001-01-01 01:00:00,Casa de Campo,ben_µg_m3,0.4
4,2001-01-01 02:00:00,Escuelas Aguirre,ben_µg_m3,9.2
...,...,...,...,...
21422011,2021-07-31 23:00:00,Casa de Campo,tol_µg_m3,0.1
21422012,2021-07-31 23:00:00,Escuelas Aguirre,tol_µg_m3,0.6
21422013,2021-07-31 23:00:00,Cuatro Caminos,tol_µg_m3,0.3
21422014,2021-07-31 23:00:00,C/ Farolillo,tol_µg_m3,0.4


### Trafico

Intensidad del tráfico desde julio 2013 (datos de los puntos de medida)

- Observaciones en periodos de 15 minutos

Contenido CSV:


|    | Nombre              | Tipo   | Descripcion                                                                                                                                                                 |
|---:|:--------------------|:-------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | idelem              | Entero | Identificación única del Punto de Medida en los sistemas de control del tráfico del Ayuntamiento de Madrid.                                                                 |
|  1 | fecha               | Fecha  | Fecha y hora oficiales de Madrid con formato yyyy-mm-dd hh:mi:ss                                                                                                            |
|  2 | identif             | Texto  | Identificador del Punto de Medida en los Sistemas de Tráfico (se proporciona por compatibilidad hacia atrás).                                                               |
|  3 | tipo_elem           | Texto  | Nombre del Tipo de Punto de Medida: Urbano o M30.                                                                                                                           |
|  4 | Intensidad          | Entero | Intensidad del Punto de Medida en el periodo de 15 minutos (vehículos/hora). Un valor negativo implica la ausencia de datos.                                                |
|  5 | ocupacion           | Entero | Tiempo de Ocupación del Punto de Medida en el periodo de 15 minutos (%).Un valor negativo implica la ausencia de datos.                                                     |
|  6 | carga               | Entero | Carga de vehículos en el periodo de 15 minutos. Parámetro que tiene en cuenta intensidad, ocupación y capacidad de la vía y establece el grado de uso de la vía de 0 a 100. |
|    |                     |        | Un valor negativo implica la ausencia de datos.                                                                                                                             |
|  7 | vmed                | Entero | Velocidad media de los vehículos en el periodo de 15 minutos (Km./h). Sólo para puntos de medida interurbanos M30. Un valor negativo implica la ausencia de datos.          |
|  8 | error               | Texto  | Indicación de si ha habido al menos una muestra errónea o sustituida en el periodo de 15 minutos.                                                                           |
|    |                     |        |       N: no ha habido errores ni sustituciones                                                                                                                              |
|    |                     |        |       E: los parámetros de calidad de alguna de las muestras integradas no son óptimos.                                                                                     |
|    |                     |        |       S: alguna de las muestras recibidas era totalmente errónea y no se ha integrado                                                                                       |
|  9 | periodo_integracion | Entero | Número de muestras recibidas y consideradas para el periodo de integración                                                                                                  |

In [ ]:
import pandas as pd
import os, zipfile, glob
zips = glob.glob("00-rawdata/Ayto Madrid/Trafico/*.zip", recursive = True)
print(len(zips),"carpetas comprimidas contienen datos de trafico")
traffic_dfs = []
num_rows = 0
#Iterar sobre todas las carpetas comprimidas para cargar los csv donde estan los datos
for zip in zips:
    with zipfile.ZipFile(zip) as z:
        for filename in z.namelist():
            if not os.path.isdir(filename):
              # read the file
              if filename.endswith('.csv'):
                df = pd.read_csv(z.open(filename),sep=';',decimal=',') #cargar csv como pandas.Dataframe
                num_rows += len(df)
                traffic_dfs.append(df)
print(len(traffic_dfs),'csv cargados')
print(num_rows,'observaciones en total')

In [36]:
df = pd.concat(traffic_dfs) #Asi son los datos de trafico dados por el Ayto. de Madrid
df.head()

,id,fecha,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion,idelem,identif,tipo
0,1001.0,2017-10-01 00:00:00,PUNTOS MEDIDA M-30,1356.0,4.0,0.0,61.0,N,5,NaN,NaN,NaN
1,1002.0,2017-10-01 00:00:00,PUNTOS MEDIDA M-30,1152.0,6.0,0.0,68.0,N,5,NaN,NaN,NaN
2,1003.0,2017-10-01 00:00:00,PUNTOS MEDIDA M-30,1404.0,5.0,0.0,71.0,N,5,NaN,NaN,NaN
3,1006.0,2017-10-01 00:00:00,PUNTOS MEDIDA M-30,1608.0,5.0,0.0,68.0,N,5,NaN,NaN,NaN
4,1009.0,2017-10-01 00:00:00,PUNTOS MEDIDA M-30,1848.0,4.0,0.0,67.0,N,5,NaN,NaN,NaN


In [37]:
df['fecha'] = pd.to_datetime(df['fecha'])
df['error'] = df['error'].astype(str)
get_data_summary(df) #Summary
# df = df.groupby("id")#.set_index("fecha").resample("60T").mean()
# df

In [ ]:
get_data_summary(df)

In [ ]:
df.set_index(['id','fecha'])#.resample("60T")

-----

In [34]:
csv = glob.glob("TRAFIC/*.csv", recursive = True) # ICAZA
print(csv)
print(len(csv))

['TRAFIC/07-2017.csv', 'TRAFIC/08-2017.csv', 'TRAFIC/09-2018.csv', 'TRAFIC/05-2020.csv', 'TRAFIC/06-2018.csv', 'TRAFIC/07-2019.csv', 'TRAFIC/04-2021.csv', 'TRAFIC/08-2019.csv', 'TRAFIC/09-2016.csv', 'TRAFIC/06-2016.csv', 'TRAFIC/05-2015.csv', 'TRAFIC/10-2017.csv', 'TRAFIC/11-2018.csv', 'TRAFIC/12-2020.csv', 'TRAFIC/03-2021.csv', 'TRAFIC/02-2020.csv', 'TRAFIC/01-2018.csv', 'TRAFIC/10-2019.csv', 'TRAFIC/11-2016.csv', 'TRAFIC/02-2015.csv', 'TRAFIC/12-2015.csv', 'TRAFIC/04-2015.csv', 'TRAFIC/07-2018.csv', 'TRAFIC/04-2020.csv', 'TRAFIC/08-2018.csv', 'TRAFIC/09-2017.csv', 'TRAFIC/06-2017.csv', 'TRAFIC/07-2016.csv', 'TRAFIC/08-2016.csv', 'TRAFIC/09-2019.csv', 'TRAFIC/05-2021.csv', 'TRAFIC/06-2019.csv', 'TRAFIC/03-2015.csv', 'TRAFIC/11-2017.csv', 'TRAFIC/02-2021.csv', 'TRAFIC/01-2019.csv', 'TRAFIC/03-2020.csv', 'TRAFIC/01-2017.csv', 'TRAFIC/10-2016.csv', 'TRAFIC/11-2019.csv', 'TRAFIC/03-2016.csv', 'TRAFIC/01-2021.csv', 'TRAFIC/02-2019.csv', 'TRAFIC/10-2020.csv', 'TRAFIC/12-2017.csv', 'TRAFIC/1

In [ ]:
dataframe = []
for i in csv:
    df = pd.read_csv(i,sep = ";", decimal = ",",error_bad_lines=False, engine='python')
    if "tipo" in df.columns:
        df = df.drop("tipo",1)
    if "identif" in df.columns:
        df = df.drop("identif",1)
    if "idelem" in df.columns:
        df = df.rename(columns={'idelem':'id'})
    dataframe.append(df)

dataframe = pd.concat(dataframe, axis=0, ignore_index=True)

In [ ]:
dataframe

In [44]:
print(dataframe.columns)
print(dataframe.shape)
dataframe.dtypes

Index(['idelem', 'fecha', 'identif', 'tipo_elem', 'intensidad', 'ocupacion',
       'carga', 'vmed', 'error', 'periodo_integracion', 'id', 'tipo',
       'idelem,fecha,identif,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion'],
      dtype='object')
(111168761, 13)


idelem                                                                                      float64
fecha                                                                                        object
identif                                                                                      object
tipo_elem                                                                                    object
intensidad                                                                                  float64
ocupacion                                                                                   float64
carga                                                                                       float64
vmed                                                                                        float64
error                                                                                        object
periodo_integracion                                                                         float64


In [45]:
print("Levels for the error variable: ",list(dataframe.error.value_counts().index))
# We will se that the error variable contains three different possible values or levels, however, this is lost when
# grouping by hours since we have chosen to use the mode for the error variable and in all cases, the mode is "N".
# This is going to make us consider deleting this variable in the future.

Levels for the error variable:  ['N', "'N'", 'E', 'S']


In [ ]:
dataframe.columns

In [ ]:
print("Idelem null values:",dataframe['idelem'].isnull().values.any())
print("Identif null values:",dataframe['identif'].isnull().values.any())
print("Tipo_elem null values:",dataframe['tipo_elem'].isnull().values.any())
print("Intensidad null values:",dataframe['intensidad'].isnull().values.any())
print("Ocupacion null values:",dataframe['ocupacion'].isnull().values.any())
print("Carga null values:",dataframe['carga'].isnull().values.any())
print("Vmed null values:",dataframe['vmed'].isnull().values.any())
print("Periodo_integracion null values:",dataframe['periodo_integracion'].isnull().values.any())

### Calidad de Aire - Comunidad de Madrid

In [12]:
#Comunidad de Madrid
estaciones_dict = {
    5 : 'ALCALÁ DE HENARES', 6 : 'ALCOBENDAS', 7 : 'ALCORCÓN', 9 : 'ALGETE',
    13 : 'ARANJUEZ', 14 : 'ARGANDA DEL REY', 16 : 'EL ATAZAR', 45 : 'COLMENAR VIEJO',
    47 : 'COLLADO VILLALBA', 49 : 'COSLADA', 58 : 'FUENLABRADA', 65 : 'GETAFE',
    67 : 'GUADALIX DE LA SIERRA', 74 : 'LEGANÉS', 80 : 'MAJADAHONDA', 92 : 'MÓSTOLES',
    103 : 'ORUSCO DE TAJUÑA', 120 : 'PUERTO DE COTES', 123 : 'RIVAS-VACIAMADRID',
    133 : 'SAN MARTÍN DE VALDEIGLESIAS', 148 : 'TORREJÓN DE ARDOZ', 161 : 'VALDEMORO',
    171 : 'VILLA DEL PRADO', 180 : 'VILLAREJO DE SALVANÉS'
}

In [13]:
zips = glob.glob("Comunidad de Madrid/*.zip", recursive = True) # Ines

In [15]:
import os, zipfile, glob
import pandas as pd
zips = glob.glob("00-rawdata/Comunidad de Madrid/*.zip", recursive = True) # Simon
aq_dfs = []
num_rows = 0
print(len(zips),"carpetas comprimidas contienen datos de calidad de aire")
#Iterar sobre todas las carpetas comprimidas para cargar los csv donde estan los datos
for zip in zips:
    if int(os.path.basename(zip).split('.zip')[0]) < 2013: #Ignorar datos de antes de 2013
        continue
    with zipfile.ZipFile(zip) as z:
        for filename in z.namelist():
            if not os.path.isdir(filename):
              # read the file
                if filename.endswith('.csv'):
                    df = pd.read_csv(z.open(filename),sep=';',decimal=',') #cargar csv como pandas.Dataframe
                    num_rows += len(df)
                    aq_dfs.append(df)
print(len(aq_dfs),'csv cargados')
print(num_rows,'observaciones en total')

17 carpetas comprimidas contienen datos de calidad de aire
104 csv cargados
476555 observaciones en total


In [16]:
df_raw = pd.concat(aq_dfs)
df = preprocess(df_raw,indicators_code_dict)
get_data_summary(df)

Dataset has 1208568 rows and 17 columns


,DataType,MissingPercent,Mean,Median,Mode,MinValue,MaxValue,NumOfUnique
ColumnName,,,,,,,,
fecha,datetime64[ns],0.000000,NaN,NaN,NaN,2013-01-01 00:00:00,2018-12-31 23:00:00,52584
ben (µg/m3),float64,0.830280,0.732713,0.3,0.1,0.0,30.1,250
co (mg/m3),float64,0.742595,0.331631,0.3,0.3,0.0,8.7,42
estacion,float64,0.000000,2.989813,2.0,1.0,1.0,14.0,7
metaparaxileno (µg/m3),float64,0.942822,0.901955,0.2,0.1,0.0,44.0,286
municipio,float64,0.000000,72.406418,65.0,5.0,5.0,180.0,24
nmhc (mg/m3),float64,0.828573,0.281334,0.2,0.1,0.0,19.54,211
no (µg/m 3),float64,0.014636,11.429508,2.0,1.0,0.0,830.0,666
no2 (µg/m 3),float64,0.014583,23.499027,14.0,4.0,0.0,465.0,286
